In [ ]:
#Data preparation

In [1]:
import os

In [2]:
from keras.preprocessing import image

In [3]:
import cv2

In [4]:
categories = ['with_mask','without_mask']

In [87]:
data=[]
for category in categories:
    path = os.path.join('C:/Users/ANUJ BISHT/Desktop/FaceMask_detector/observations-master/experiements/dest_folder/train',category)
    # print(path)
    # for everytime
    # read every file
     
    # index like (with_mack-0 , without_mask-1)
    label = categories.index(category)
    for file in os.listdir(path):
        
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        #resize krre h image ko coz VGG16 take this size image only
        img = cv2.resize(img,(224,224))
        
        #print(img.shape)
        # humae end mae ek dataset bana h jismae 2 column hongae ((numpyarray) of image),Label
        data.append([img,label])

In [88]:
len(data)

1650

In [89]:
#because our model will develope an bias
import random 
random.shuffle(data)

In [90]:
data

[[array([[[ 81, 110, 124],
          [ 78, 104, 118],
          [ 83, 105, 120],
          ...,
          [  7,   8,  24],
          [  6,   9,  24],
          [  4,   9,  24]],
  
         [[ 82, 112, 126],
          [ 79, 105, 119],
          [ 79, 100, 116],
          ...,
          [  7,   9,  19],
          [  8,  10,  21],
          [  6,  11,  21]],
  
         [[ 79, 108, 123],
          [ 82, 107, 123],
          [ 83, 104, 121],
          ...,
          [  8,  10,  15],
          [  4,   7,  12],
          [  4,   9,  12]],
  
         ...,
  
         [[ 43,  62,  79],
          [ 43,  66,  77],
          [ 37,  62,  71],
          ...,
          [  5,   8,  13],
          [  3,   5,   9],
          [  4,   4,   7]],
  
         [[ 42,  62,  79],
          [ 41,  63,  75],
          [ 37,  61,  70],
          ...,
          [  4,   5,   9],
          [  4,   5,   9],
          [  4,   5,   9]],
  
         [[ 43,  63,  80],
          [ 39,  63,  75],
          [ 35,  59,  68

In [55]:
len(data)

1508

In [91]:
# x & y ko alag-alag karna h , and convert them into numpyarray
X = [] # we have to make it 2-D array
y = [] # 1-D array
for features,label in data:
    X.append(features)
    y.append(label)

In [92]:
import numpy as np

In [93]:
X = np.array(X)
y = np.array(y)

In [94]:
# to standardize x
X = X/255

In [95]:
# x_train and y_train
# data is ready 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2) 

In [38]:
# humara CNN architectur create karengae
# we are doing transfer learning

In [96]:
from keras.applications.vgg16 import VGG16

In [97]:
vgg = VGG16() # It is a functional model and we want Sequential model

In [17]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [98]:
#to make model according to our need we have to remove the last line and add our own 
from keras import Sequential

In [99]:
model = Sequential()

In [100]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [101]:
# now we have to freeze every layerso that on training that weight does-not updates
# now we cannot upgrade model during training process
for layer in model.layers:
    layer.trainable=False
# all trainable become un-trainable

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [102]:
from keras.layers import Dense

In [103]:
model.add(Dense(1,activation='sigmoid'))

In [27]:
# 4096 honge 1 iska khud ka hoga bias 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [104]:
# we can decrease learning rate during transfer learning generally (0.0001)
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [105]:
# THIS IS WITHOUT DATA AUGUMENTATION
model.fit(X_train , y_train, epochs=5, validation_data = (X_test, y_test))

Epoch 1/5
42/42 [==============================] - 236s 6s/step - loss: 0.5845 - accuracy: 0.7485 - val_loss: 0.3996 - val_accuracy: 0.8939
Epoch 2/5
42/42 [==============================] - 230s 5s/step - loss: 0.3322 - accuracy: 0.9106 - val_loss: 0.2885 - val_accuracy: 0.9152
Epoch 3/5
42/42 [==============================] - 231s 6s/step - loss: 0.2427 - accuracy: 0.9386 - val_loss: 0.2362 - val_accuracy: 0.9182
Epoch 4/5
42/42 [==============================] - 257s 6s/step - loss: 0.1990 - accuracy: 0.9455 - val_loss: 0.1963 - val_accuracy: 0.9485
Epoch 5/5
42/42 [==============================] - 243s 6s/step - loss: 0.1730 - accuracy: 0.9568 - val_loss: 0.1734 - val_accuracy: 0.9606


In [132]:
# now we use the model for detecting face in video
# first video streaming then prediction
cap = cv2.VideoCapture(0) # 0 for webcam

In [131]:
while True:
    # ret(True/False) ||| frame(image)
    ret, frame = cap.read() 
        
    # call the detection method
    img = cv2.resize(frame,(224,224))
    
    y_pred = detect_face_mask(img) 
    
    coods = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    
    for x,y,w,h in coods:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
        
    # draw_label(frame,"Face mask Detection",(30,30),(255,0,0))
    # call face detection method
        
    if y_pred == 0:
        draw_label(frame,"Mask",(30,30),(0,255,0))
    else:
        draw_label(frame,"No Mask",(30,30),(0,0,255))
        
    cv2.imshow("window",frame)
    #break condition
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
        
cv2.destroyAllWindows()

1/1 [==============================] - 0s 241ms/step


1/1 [==============================] - 0s 216ms/step


In [125]:
def detect_face_mask(img):
    y_pred = model.predict(img.reshape(1,224,224,3))
    ans = round(y_pred[0][0])
    return ans

In [126]:
sample1 = cv2.imread('C:/Users/ANUJ BISHT/Desktop/FaceMask_detector/observations-master/experiements/dest_folder/train/without_mask/pm2.jpg')
sample1 = cv2.resize(sample1,(224,224))

In [127]:
detect_face_mask(sample1)

1/1 [==============================] - 0s 256ms/step


1

In [111]:
def draw_label(img, text, pos, bg_color):
    
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1 , cv2.FILLED)
    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[1] + text_size[0][1] - 2
    
    #image,position, kahan por, background color, thickness
    cv2.rectangle(img, pos,(end_x,end_y),bg_color,cv2.FILLED)
    
    # to write text
        # image, kya text, position , font , scale , color , which type of line
        
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

In [71]:
while True:
    ret, frame = cap.read() # ret(True/False) ||| frame(image)
    
    #grey scale image
    coods = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    
    for x,y,w,h in coods:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
    
    cv2.imshow("window",frame)
    #break condition
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
        
cv2.destroyAllWindows()

In [112]:
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [113]:
def detect_face(img):
    coods = haar.detectMultiScale(img)
    return coods